# Guided Project: Analyzing Startup Fundraising Deals from Crunchbase

### Introduction
In this project, we will be using pandas to analyze startup investments from Chrunchbase. The data set of investments we'll be exploring is current as of October 2013, which can be downloaded from [Github](https://github.com/datahoarder/crunchbase-october-2013/blob/master/crunchbase-investments.csv).

First we need to read the data set into dataframes using 5,000 row chunks. This is to ensure thateach chunk consumes much less than 10 megabytes of memory.

In [1]:
import pandas as pd
pd.options.display.max_columns = 99
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

Now within those chunks, we need to find each column's missing value counts.

In [2]:
mv_list = []
for chunk in chunk_iter:
    mv_list.append(chunk.isnull().sum())
    
combined_mv_vc = pd.concat(mv_list)
unique_combined_mv_vc = combined_mv_vc.groupby(combined_mv_vc.index).sum()
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

Now we find each column's memory footprint.

In [3]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
counter = 0
series_memory_fp = pd.Series(dtype='float64')
for chunk in chunk_iter:
    if counter == 0:
        series_memory_fp = chunk.memory_usage(deep=True)
    else:
        series_memory_fp += chunk.memory_usage(deep=True)
    counter += 1
    
series_memory_fp = series_memory_fp.drop('Index')
series_memory_fp

company_permalink         4057788
company_name              3591326
company_category_code     3421104
company_country_code      3172176
company_state_code        3106051
company_region            3411585
company_city              3505926
investor_permalink        4980548
investor_name             3915666
investor_category_code     622424
investor_country_code     2647292
investor_state_code       2476607
investor_region           3396281
investor_city             2885083
funding_round_type        3410707
funded_at                 3542185
funded_month              3383584
funded_quarter            3383584
funded_year                422960
raised_amount_usd          422960
dtype: int64

Lastly, we find the total memory footprint with all of the chunks combined.

In [4]:
series_memory_fp.sum() / (1024*1024)

56.9876070022583

The total memory footprint is 56.99 megabytes of memory.

Now we will find columns that we can drop because they aren't useful for our analysis.

In [5]:
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [6]:
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = chunk.columns.drop(drop_cols)

We dropped the columns investor_permalink, company_permalink and investor_category_code because they are not useful information we will use in our analysis.

Below are columns we can use for our analysis:

In [7]:
keep_cols.tolist

<bound method IndexOpsMixin.tolist of Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')>

### Selecting Data Types
Let's first determine which columns shift types across chunks.

In [8]:
col_types = {}
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

for chunk in chunk_iter:
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

We'll identify the types for each column and then identify the numeric columns we can represent using more space efficient types.

In [9]:
uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'raised_amount_usd': {'float64'}}

In [10]:
col_dtypes = {
    "company_name": "category", "company_category_code": "category", "company_country_code": "category",
    "company_state_code": "category", "company_city": "category", "investor_name": "category",
    "investor_category_code": "category", "investor_country_code": "category", "investor_state_code": "category",
    "investor_city": "category", "funding_round_type": "category", "raised_amount_usd": "float64"
}
    
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=keep_cols, dtype =col_dtypes, encoding='ISO-8859-1')
    
mem_usage = []
for chunk in chunk_iter:
    mem_usage.append(chunk.memory_usage(deep=True).sum() / 1024 ** 2)
sum(mem_usage)    

22.660161018371582

If we convert the object types to category types, we will make the memory usage more efficient. After converting, we decreased the total memory footprint from 56.99 megabytes to 22.66 megabytes. 

### Loading Chunks into SQLite

In [11]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=keep_cols, dtype =col_dtypes, encoding='ISO-8859-1')

for chunk in chunk_iter:
    chunk.to_sql("investments", conn, if_exists='append', index=False)